In [ ]:
from statsmodels.api import OLS
from statsmodels.tsa.stattools import OLS 
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np
from zipline.finance import commission, slippage
from zipline.api import(    symbol,
                            get_datetime,
                            order_target_percent,
                            schedule_function,
                            date_rules,
                            time_rules,
                            attach_pipeline,
                            pipeline_output,
                            set_commission,
                            set_slippage,
                            get_open_orders,
                            cancel_order,
                            order_target)

def initialize(context):
    context.securities = ["ALBK",'UNIONBANK',"CANBK","SYNDIBANK","AUROPHARMA","LUPIN","BIOCON","WOCKPHARMA"]
    context.pairs = [("ALBK",'UNIONBANK'),("CANBK","SYNDIBANK"),("AUROPHARMA","LUPIN"),("BIOCON","WOCKPHARMA")]#manually uploaded pairs 

    context.params = {"zscore_threshold":-4,
                     "long_spread_stop":-3,
                     "short_spread_stop":3,
                     "trade_frequency":60,
                     "look_back_period":30,
                     "risk_per_trade":0.01,
                     "stop_loss":1,
                     }

 
    context.signal_status = {}# to store information about the pairs wich are we currenlty have position in 
    context.order_df = pd.DataFrame({"signal":0},index =context.securities)# dataframe creation with stock names as index 
    schedule_function(run_stratergy,date_rules.every_day(),time_rules.market_open())

    
    
def run_stratergy(context,data):
    for x,y in context.pairs:
        generate_signals(context,data,x,y)
    rebalance(context,data)   




def generate_signals(context,data,x_name,y_name):
    x = data.history(symbol(x_name),"close",context.params["look_back_period"],"1d")
    y = data.history(symbol(y_name),"close",context.params["look_back_period"],"1d")
    model = OLS(y,x).fit()
    spread = y - (model.params[0] * x)
    beta = model.params[0]
    adf = adfuller(spread)
    adf_tstat = adf[0]
    threshold = adf[4]['5%']
    combine_name = str(x_name) + str(y_name)# this varible will be used as key to store all the information about the pairs while entring the position 
    
    # The threshold value was incorrect
    if adf_tstat < threshold:#checking whether the spread is sationary or not 
        zscore = (spread[-1] - np.mean(spread))/np.std(spread)
        #position = (position_size(context,data,spread) * spread[-1])/context.params['portfolio']# i am not sure it the calculation done in this function is making sense or not 
        position = 100# so for testing phase i am keeping fix position for a pair to check other code are working or not 
        
        if combine_name not in context.signal_status:#this code block is for the pairs we dont have any open positions
                
            
            if zscore >= 2 and zscore <= 2.9:
                
                #short the spread
                #updating dataframe as well 
                # if i enter a positions in pair i record the neccery variables in dict to be used while exiting the position 
                context.signal_status[combine_name] = ["short",beta*position,-1 * position,beta]#attaching the list to dict key combine_name in signal_status
                context.order_df.loc[x_name]["signal"] = context.order_df.loc[x_name]["signal"] + (beta*position)#updatig the dataframe with also taking care of previous positions if any 
                context.order_df.loc[y_name]["signal"] = context.order_df.loc[y_name]["signal"] + (-1*position)                
            
            elif zscore <= -2 and zscore >= -2.9:
                
                #long the spread
                context.signal_status[combine_name] = ["long",-beta * position,1 * position,beta]
                context.order_df.loc[x_name]["signal"] = context.order_df.loc[x_name]["signal"] + (-beta*position)
                context.order_df.loc[y_name]["signal"] = context.order_df.loc[y_name]["signal"] + (1*position)  
                
        elif combine_name in context.signal_status:# this code block is for the pairs we have already taken positions
            
            spread = y - (context.signal_status[combine_name][3] * x)#caculating the spread with recorded beta value  while entring the position in the pair 
            zscore = (spread[-1] - np.mean(spread))/np.std(spread)
            
            if context.signal_status[combine_name][0] == "short":#checking if we are short the pair 
                
                if zscore >= 3: # stop loss condition for "short" positions
                    #subtracting the same amount of shares  from dataframe  while entering the position into a pair 
                    context.order_df.loc[x_name]["signal"] = context.order_df.loc[x_name]["signal"] + (-context.signal_status[combine_name][1])
                    context.order_df.loc[y_name]["signal"] = context.order_df.loc[y_name]["signal"] + (-(context.signal_status[combine_name][2]))    
                    context.signal_status.pop(combine_name)

                    
                elif zscore <= 0: #target booking condition for "short" positions
                    
                    context.order_df.loc[x_name]["signal"] = context.order_df.loc[x_name]["signal"] + (-context.signal_status[combine_name][1])
                    context.order_df.loc[y_name]["signal"] = context.order_df.loc[y_name]["signal"] + (-(context.signal_status[combine_name][2]))
                    context.signal_status.pop(combine_name)

            
            elif context.signal_status[combine_name][0] == "long":#checking if are long the pair 
                
            
                if zscore <= -3:# stop loss condition for "long" positions
                    
                    context.order_df.loc[x_name]["signal"] = context.order_df.loc[x_name]["signal"] + (-(context.signal_status[combine_name][1]))#converting a negetive position  size in the dict to positive to sqaure off the position
                    context.order_df.loc[y_name]["signal"] = context.order_df.loc[y_name]["signal"] + (-context.signal_status[combine_name][2])
                    context.signal_status.pop(combine_name)
                    print('Stop Loss %s' % combine_name)
                    print('zscore is %.2f' % zscore)
                    
                elif zscore >= 0: #target booking condition for "long" positions
                    context.order_df.loc[x_name]["signal"] = context.order_df.loc[x_name]["signal"] + (-(context.signal_status[combine_name][1]))
                    context.order_df.loc[y_name]["signal"] = context.order_df.loc[y_name]["signal"] + (-context.signal_status[combine_name][2])                    
                    context.signal_status.pop(combine_name)

                    
    elif combine_name in context.signal_status:
        #this code block is for if we have the position in a pair but pair no more satisfies the adf_test so we exit 
        
        if context.signal_status[combine_name][0] == "short":
                
            context.order_df.loc[x_name]["signal"] = context.order_df.loc[x_name]["signal"] + (-context.signal_status[combine_name][1])
            context.order_df.loc[y_name]["signal"] = context.order_df.loc[y_name]["signal"] + (-(context.signal_status[combine_name][2]))
            context.signal_status.pop(combine_name)
            
        elif context.signal_status[combine_name][0] == "long":
            context.order_df.loc[x_name]["signal"] = context.order_df.loc[x_name]["signal"] + (-(context.signal_status[combine_name][1]))
            context.order_df.loc[y_name]["signal"] = context.order_df.loc[y_name]["signal"] + (-context.signal_status[combine_name][2])
            context.signal_status.pop(combine_name)

          
                

def rebalance(context,data):
    print(context.order_df)
    for index,row in context.order_df.iterrows():#itrating through the rows of the data frame to ascess name and position
        order_target(symbol(index),row[0])
 


